In [1]:
import glacierml as gl
import os
import pandas as pd
import numpy as np
pd.set_option('display.max_column',None)
import path_manager as pm
[
        home_path, data_path, RGI_path, glathida_path, 
        ref_path, coregistration_testing_path, 
        arch_test_path, LOO_path
] = pm.set_paths()

In [2]:
RGI = gl.load_RGI(RGI_path)
glathida = pd.read_csv(os.path.join(glathida_path, 'T.csv'))
glathida = glathida.dropna(subset = ['MEAN_THICKNESS'])
glathida = glathida.reset_index().drop('index',axis = 1)

In [3]:
# if not os.path.exists(
#     os.path.join(
#         data_path, 'matched.pkl'
# )):
#     df = gl.match_GlaThiDa_RGI_index(
#     RGI,glathida,verbose = True, useMP = True
# )
    
#     df.to_pickle(os.path.join(data_path,'matched.pkl'))
    
# else:
#     print('data already matched')

In [4]:
def get_filtered_RGI(glathida_ll, RGI, max_distance=500):
    # Filter RGI glaciers within a rough bounding box (approximate filter to reduce computation)
    lat, lon = glathida_ll
    lat_min, lat_max = lat - 5, lat + 5  # Approx 5-degree latitude window
    lon_min, lon_max = lon - 5, lon + 5  # Approx 5-degree longitude window
    return RGI[(RGI['CenLat'] >= lat_min) & (RGI['CenLat'] <= lat_max) &
               (RGI['CenLon'] >= lon_min) & (RGI['CenLon'] <= lon_max)]

from joblib import Parallel, delayed
from tqdm import tqdm
import pandas as pd

# Haversine function for fast distance calculation
def haversine(lat1, lon1, lat2, lon2):
    from math import radians, sin, cos, sqrt, atan2
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    radius_earth_km = 6371  # Radius of the Earth in kilometers
    return radius_earth_km * c

def get_id_and_distance(RGI, glathida, i):
    glathida_ll = (glathida.loc[i].LAT, glathida.loc[i].LON)

    # Use filtered RGI to reduce computation
    RGI_filtered = get_filtered_RGI(glathida_ll, RGI)

    distances = RGI_filtered.apply(
        lambda row: haversine(row.CenLat, row.CenLon, glathida_ll[0], glathida_ll[1]),
        axis=1
    )
    centroid_distance = distances.min()
    min_indices = np.where(distances == centroid_distance)

    RGI_index = pd.Series(min_indices[0], name='RGI_indexes')
    
    if len(RGI_index) == 1:
        RGI_id_match = RGI_filtered['RGIId'].iloc[RGI_index.iloc[0]]
    else:
        RGI_id_match = -1
        centroid_distance = -1

    return RGI_id_match, centroid_distance

# Function to update glathida dataframe
def update_glathida(RGI, glathida, i):
    RGI_id_match, centroid_distance = get_id_and_distance(RGI, glathida, i)
    return i, RGI_id_match, centroid_distance

# Parallel processing with progress bar
def parallel_with_progress_bar(RGI, glathida, num_workers=4):
    with tqdm(total=len(glathida), desc="Processing glaciers", unit="glacier") as pbar:
        output = Parallel(n_jobs=num_workers)(
            delayed(update_glathida)(RGI, glathida, i) for i in glathida.index
        )
        
        # Update glathida with matched RGIId and centroid distance
        for i, RGI_id_match, centroid_distance in output:
            glathida.at[i, 'RGIId'] = RGI_id_match
            glathida.at[i, 'RGI Centroid Distance'] = centroid_distance
            pbar.update(1)

# Running the parallel processing with progress bar
parallel_with_progress_bar(RGI, glathida)

# Now glathida has been updated with RGIId and centroid distance



Processing glaciers: 100%|███████████████| 500/500 [01:13<00:00,  6.81glacier/s]


In [5]:
glathida = glathida.drop(glathida[glathida['RGIId'] == -1].index)

In [7]:
glathida.to_pickle(os.path.join(data_path,'matched.pkl'))